# CS246 - Colab 1
## Wordcount in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you run successfully the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter we want to count the total number of (non-unique) words that start with a specific letter. In your implementation **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all the words **starting** with a non-alphabetic character.

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [10]:
df = spark.read.csv("pg100.txt")
df.show()

+--------------------+------------------+
|                 _c0|               _c1|
+--------------------+------------------+
|The Project Guten...|                by|
| William Shakespeare|              null|
|This eBook is for...|              null|
|almost no restric...|   give it away or|
|re-use it under t...|              null|
|with this eBook o...|              null|
|** This is a COPY...|  Details Below **|
|**     Please fol...|              null|
|Title: The Comple...|              null|
|Author: William S...|              null|
|Posting Date: Sep...| 2011 [EBook #100]|
|Release Date: Jan...|              1994|
|   Language: English|              null|
|*** START OF THIS...|              null|
|Produced by World...|              Inc.|
|This is the 100th...|               and|
|is presented in c...|              Inc.|
|Library of the Fu...|              null|
|often releases Et...|              null|
|         Shakespeare|              null|
+--------------------+------------

In [0]:
# CODE
import re
df=df.toPandas()

In [12]:
# HERE
df["_c2"]=df["_c0"].apply(lambda x:re.findall(r"^[a-z]+",x.lstrip().lower()))
df["_c3"]=df["_c2"].apply(lambda x:int(x!=[]))
df=df[df["_c3"]==1]
df.head(20)

,_c0,_c1,_c2,_c3
0,The Project Gutenberg EBook of The Complete Wo...,by,[the],1
1,William Shakespeare,None,[william],1
2,This eBook is for the use of anyone anywhere a...,None,[this],1
3,almost no restrictions whatsoever. You may co...,give it away or,[almost],1
4,re-use it under the terms of the Project Guten...,None,[re],1
5,with this eBook or online at www.gutenberg.org,None,[with],1
8,Title: The Complete Works of William Shakespeare,None,[title],1
9,Author: William Shakespeare,None,[author],1
10,Posting Date: September 1,2011 [EBook #100],[posting],1
11,Release Date: January,1994,[release],1


In [0]:
df["_c4"]=df["_c2"].apply(lambda x:x[0][0])

In [14]:
pd.DataFrame(pd.value_counts(df["_c4"]))

,_c4
t,15568
a,14499
w,8092
s,7640
i,7015
b,6520
h,5506
c,5416
f,4950
m,4736


Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!